In [1]:
"""
Simple Inference Script of EfficientDet-Pytorch
"""
import time

import torch
#from torch.backends import cudnn

from backbone import EfficientDetBackbone
import cv2
import numpy as np

from efficientdet.utils import BBoxTransform, ClipBoxes
from utils.utils import preprocess, invert_affine, postprocess

In [124]:
compound_coef = 0
force_input_size = None  # set None to use default size
img_path = '/Users/wsarguroh001/Documents/Videos/TownCentre/TownCentre3950.jpg'

# replace this part with your project's anchor config
anchor_ratios = [(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)]
anchor_scales = [2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]

threshold = 0.2
iou_threshold = 0.2

use_cuda = False
use_float16 = False
#cudnn.fastest = True
#cudnn.benchmark = True

#obj_list = ['person']
obj_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
            'fire hydrant', '', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
            'cow', 'elephant', 'bear', 'zebra', 'giraffe', '', 'backpack', 'umbrella', '', '', 'handbag', 'tie',
            'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
            'skateboard', 'surfboard', 'tennis racket', 'bottle', '', 'wine glass', 'cup', 'fork', 'knife', 'spoon',
            'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut',
            'cake', 'chair', 'couch', 'potted plant', 'bed', '', 'dining table', '', '', 'toilet', '', 'tv',
            'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink',
            'refrigerator', '', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
            'toothbrush']

# tf bilinear interpolation is different from any other's, just make do
input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536]
input_size = input_sizes[compound_coef] if force_input_size is None else force_input_size
ori_imgs, framed_imgs, framed_metas = preprocess(img_path, max_size=input_size)

In [125]:
if use_cuda:
    x = torch.stack([torch.from_numpy(fi).cuda() for fi in framed_imgs], 0)
else:
    x = torch.stack([torch.from_numpy(fi) for fi in framed_imgs], 0)

x = x.to(torch.float32 if not use_float16 else torch.float16).permute(0, 3, 1, 2)

model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=len(obj_list),
                             ratios=anchor_ratios, scales=anchor_scales)
model.load_state_dict(torch.load(f'weights/efficientdet-d{compound_coef}.pth'))
model.requires_grad_(False)
model.eval()

if use_cuda:
    model = model.cuda()
if use_float16:
    model = model.half()

with torch.no_grad():
    features, regression, classification, anchors = model(x)

    regressBoxes = BBoxTransform()
    clipBoxes = ClipBoxes()

    out = postprocess(x,
                      anchors, regression, classification,
                      regressBoxes, clipBoxes,
                      threshold, iou_threshold)

In [4]:
def display(preds, imgs, imshow=True, imwrite=False):
    for i in range(len(imgs)):
        if len(preds[i]['rois']) == 0:
            continue

        for j in range(len(preds[i]['rois'])):
            (x1, y1, x2, y2) = preds[i]['rois'][j].astype(np.int)
            
            obj = obj_list[preds[i]['class_ids'][j]]
            score = float(preds[i]['scores'][j])

            if obj == 'person':
                cv2.putText(imgs[i], '{}, {:.3f}'.format(obj, score),
                            (x1, y1 + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            (255, 255, 0), 1)
                cv2.rectangle(imgs[i], (x1, y1), (x2, y2), (255, 255, 0), 2)

        if imshow:
            cv2.imshow('img', imgs[i])
            cv2.waitKey(0)

        if imwrite:
            cv2.imwrite(f'test/img_inferred_d{compound_coef}_this_repo_{i}.jpg', imgs[i])

In [126]:
out = invert_affine(framed_metas, out)

In [16]:
display(out, ori_imgs, imshow=False, imwrite=True)

In [8]:
print('running speed test...')
with torch.no_grad():
    print('test1: model inferring and postprocessing')
    print('inferring image for 10 times...')
    t1 = time.time()
    for _ in range(10):
        _, regression, classification, anchors = model(x)

        out = postprocess(x,
                          anchors, regression, classification,
                          regressBoxes, clipBoxes,
                          threshold, iou_threshold)
        out = invert_affine(framed_metas, out)

    t2 = time.time()
    tact_time = (t2 - t1) / 10
    print(f'{tact_time} seconds, {1 / tact_time} FPS, @batch_size 1')

    # uncomment this if you want a extreme fps test
    # print('test2: model inferring only')
    # print('inferring images for batch_size 32 for 10 times...')
    # t1 = time.time()
    # x = torch.cat([x] * 32, 0)
    # for _ in range(10):
    #     _, regression, classification, anchors = model(x)
    #
    # t2 = time.time()
    # tact_time = (t2 - t1) / 10
    # print(f'{tact_time} seconds, {32 / tact_time} FPS, @batch_size 32')

running speed test...
test1: model inferring and postprocessing
inferring image for 10 times...
0.7449088096618652 seconds, 1.3424461988225482 FPS, @batch_size 1


# Running Efficient Det on Video

In [9]:
import cv2

In [11]:
#Keep Video at root folder and create a folder named 'Videos' at root
# Read Video File and Write Image Files

cap = cv2.VideoCapture('TownCentreXVID.avi')

i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('Videos/TownCentre'+str(i)+'.jpg',frame)
    i+=1

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [84]:
def area_bbox(x1, y1, x2, y2):
    return (x2 - x1)*(y2 - y1)

def is_too_close(p1, p2):
    x11, y11, x12, y12 = p1
    x21, y21, x22, y22 = p2
    #total_region_coords 
    X1 = min(x11, x21)
    Y1 = min(y11, y21)
    X2 = max(x12, x22)
    Y2 = max(y12, y22)
    
    ratio = area_bbox(X1, Y1, X2, Y2)/sum([area_bbox(x11, y11, x12, y12), area_bbox(x21, y21, x22, y22)])
    
    if ratio<1.2:
        return True
    else:
        return False

In [127]:
new_out = {}
new_out['rois'] = []
new_out['scores'] = []
for i in range(len(out[0]['rois'])):
    if out[0]['class_ids'][i] == 0:
        new_out['rois'].append(out[0]['rois'][i])
        new_out['scores'].append(out[0]['scores'][i])

In [128]:
new_out['is_too_close'] = []
for i in range(len(new_out['rois'])):
    test = any([is_too_close(new_out['rois'][i], p) for p in new_out['rois'][:i] + new_out['rois'][i+1:]])
    new_out['is_too_close'].append(test)

In [97]:
def display_modified(new_out, img):
    #for i in range(len(imgs)):
    if len(new_out['rois']) == 0:
        return 0

    for j in range(len(new_out['rois'])):
        (x1, y1, x2, y2) = new_out['rois'][j].astype(np.int)

        if new_out['is_too_close'][j]:
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        else:
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        cv2.imwrite('test.jpg', img)

In [129]:
display_modified(new_out, ori_imgs[0])